In [1]:
import os, json
from Tint_Matlab import *
import struct
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import SignalProcessing as sp
%matplotlib inline
import re
%load_ext line_profiler
import profile
import time
import mmap
import contextlib
import numpy.distutils.system_info as sysinfo
from pyfftw.interfaces import scipy_fftpack as fftw

In [2]:
#bin_filename = 'C:\\Users\\Geoffrey Barrett\\Desktop\\RAW Data\\20170823-3-CYLINDEROPENFIELD_Mult.bin'
# bin_filename = 'C:\\Users\\Geoffrey Barrett\\Desktop\\RAW Data\\20170823-2-CYLINDEROPENFIELD.bin'
#bin_filename = 'C:\\Users\\Taub Institute\\Desktop\\RAW Data\\20170823-3-CYLINDEROPENFIELD_Mult.bin'
#bin_filename = 'C:\\Users\\Taub Institute\\Desktop\\RAW Data\\20170823-2-CYLINDEROPENFIELD.bin'
# bin_filename = "C:\\Users\\Taub Institute\\Desktop\\RAW Data\\ConvFromOtherDaq\\20170829-1-CYLINDEROPENFIELD.bin"
#bin_filename = "C:\\Users\\Geoffrey Barrett\\Desktop\\RAW Data\\ConvFromOtherDaq\\20170829-1-CYLINDEROPENFIELD.bin"
bin_filename = 'E:\\Apollo_D_Drive\\ApolloKlusta\\Processed\\Raw30Min - Copy\\20170927-RAW-30MIN.bin'
bin_directory = os.path.dirname(bin_filename)
session = os.path.basename(os.path.splitext(bin_filename)[0])
set_filename = os.path.join(bin_directory, '%s.set' % session)
pos_filename = os.path.join(bin_directory, '%s.pos' % session)

In [3]:
def samples_to_array(A, channels=[]):
    """This will take data matrix A, and convert it into a numpy array, there are three samples of
    64 channels in this matrix, however their channels do need to be re-mapped"""
    
    
    if channels == []:
        channels = np.arange(64)+1
    else:
        channels = np.asarray(channels)
    
    A = np.asarray(A)
    
    sample_num = int(len(A)/64) # get the sample numbers
    
    sample_array = np.zeros((len(channels),sample_num)) # creating a 64x3 array of zeros (64 channels, 3 samples)
    
    '''
    for i in range(3):
        # there are three samples
        
        current_sample = A[:64] # defines current samples
        
        for k in range(64):
            
            sample_array[k,i] = current_sample[get_remap_chan(k+1)]
        
        A = A[64:]  # discards the used samples
    '''
    
    for i, channel in enumerate(channels):
        sample_array[i,:] = A[get_sample_indices(channel, sample_num)]
            
    return sample_array

def get_sample_indices(channel_number, samples):
    remap_channel = get_remap_chan(channel_number)
    
    indices_scalar = np.multiply(np.arange(samples), 64)
    sample_indices = indices_scalar + np.multiply(np.ones(samples), remap_channel)
    
    #return np.array([remap_channel, 64 + remap_channel, 64*2 + remap_channel])
    return (indices_scalar + np.multiply(np.ones(samples), remap_channel)).astype(int)
    

def get_remap_chan(chan_num):
    """There is re-mapping, thus to get the correct channel data, you need to incorporate re-mapping
    input will be a channel from 1 to 64, and will return the remapped channel"""
    
    remap_channels = np.array([32, 33, 34, 35, 36, 37, 38, 39, 0, 1, 2, 3, 4, 5, 
                      6, 7, 40, 41, 42, 43, 44, 45, 46, 47, 8, 9, 10, 11, 
                      12, 13, 14, 15, 48, 49, 50, 51, 52, 53, 54, 55, 16, 17, 
                      18, 19, 20, 21, 22, 23, 56, 57, 58, 59, 60, 61, 62, 63, 
                      24, 25, 26, 27, 28, 29, 30, 31])
    
    return remap_channels[chan_num-1]
            
            
def find_sub(string, sub):
    '''finds all instances of a substring within a string and outputs a list of indices'''
    result = []
    k = 0
    while k < len(string):
        k = string.find(sub, k)
        if k == -1:
            return result
        else:
            result.append(k)
            k += 1  # change to k += len(sub) to not search overlapping results
    return result


def get_active_tetrode(set_filename):
    """in the .set files it will say collectMask_X Y for each tetrode number to tell you if 
    it is active or not. T1 = ch1-ch4, T2 = ch5-ch8, etc."""
    active_tetrode = []
    active_tetrode_str = 'collectMask_'
    
    with open(set_filename) as f:
        for line in f:

            # collectMask_X Y, where x is the tetrode number, and Y is eitehr on or off (1 or 0)
            if active_tetrode_str in line:
                tetrode_str, tetrode_status = line.split(' ')
                if int(tetrode_status) == 1:
                    # then the tetrode is saved
                    tetrode_str.find('_')
                    tet_number = int(tetrode_str[tetrode_str.find('_')+1:])
                    active_tetrode.append(tet_number)
                    
    return active_tetrode

def get_active_eeg(set_filename):
    """This will return a dictionary (cative_eeg_dict) where the keys 
    will be eeg channels from 1->64 which will represent the eeg suffixes (2 = .eeg2, 3 = 2.eeg3, etc)
    and the key will be the channel that the EEG maps to (a channel from 0->63)"""
    active_eeg = []
    active_eeg_str = 'saveEEG_ch'
    
    eeg_map = []
    eeg_map_str = 'EEG_ch_'
    
    active_eeg_dict = {}
    
    with open(set_filename) as f:
        for line in f:
            
            if active_eeg_str in line:
                 # saveEEG_ch_X Y, where x is the eeg number, and Y is eitehr on or off (1 or 0)
                _, status = line.split(' ')
                active_eeg.append(int(status))
            elif eeg_map_str in line:
                #EEG_ch_X Y
                _, chan = line.split(' ')
                eeg_map.append(int(chan))
                
                
        # active_eeg = np.asarray(active_eeg)
        # eeg_map = np.asarray(eeg_map)
    
    for i, status in enumerate(active_eeg):
        if status == 1:
            active_eeg_dict[i+1] = eeg_map[i] - 1
        
    return active_eeg_dict


def is_egf_active(set_filename):
    
    active_egf_str = 'saveEGF'
    
    with open(set_filename) as f:
        for line in f:
            
            if active_egf_str in line:
                _, egf_status = line.split(' ')
                
                if int(egf_status) == 1:
                    return True
                
        return False

    
def get_Fs(set_filename):
    
    fs_str = 'rawRate'
    
    with open(set_filename) as f:
        for line in f:
            
            if fs_str in line:
                _, Fs = line.split(' ')
                
                try:
                    return int(Fs)
                except:
                    return float(Fs)
    return []
   
    
def create_eeg(filename, data, Fs):
    # data is given in int16, and already has been notch filtered
    
    if os.path.exists(filename):
        return
    
    Fs_EEG = 250  # sampling rate of .EEG files
    
    data = sp.Filtering().iirfilt(bandtype='low', data=data, Fs=Fs, Wp=Fs_EEG / 2, order=6, 
                                  automatic=0, Rp=3, As=60, filttype='butter', showresponse=0)
    
    ##################################################################################################
    # ---------------------------Downsample to produce the proper EEG------------------------
    ##################################################################################################
    
    data = data[:, 0::int(Fs / Fs_EEG)]
    
    # ensuring the appropriate range of the values
    data[np.where(data > 32767)] = 32767
    data[np.where(data < -32768)] = -32768
    
    data = int16toint8(data) # converting from 16 bits to 8 bits, 
    
    # ensurinig appropriate range of values (this shouldn't be necessary)
    data[np.where(data > 127)] = 127
    data[np.data(data < -128)] = -128
    
    ##################################################################################################
    # ---------------------------Writing the EEG Data-------------------------------------------
    ##################################################################################################
    
    write_eeg(filename, data, Fs_EEG)
    
def create_egf(filename, data, Fs):
    
    if os.path.exists(filename):
        return
    
    Fs_EGF = 4.8e3  # sampling rate of .EGF files
    
    data = sp.Filtering().iirfilt(bandtype='low', data=data, Fs=Fs, Wp=Fs_EGF / 2, order=6, 
                                  automatic=0, Rp=3, As=60, filttype='butter', showresponse=0)
    
    ##################################################################################################
    # ---------------------------Downsample to produce the proper EGF Data------------------------
    ##################################################################################################
    
    data = data[:, 0::int(Fs / Fs_EGF)]
    
    # ensuring the appropriate range of the values
    data[np.where(data > 32767)] = 32767
    data[np.where(data < -32768)] = -32768
    
    # data is already in int16 which is what the final unit should be in
    
    ##################################################################################################
    # ---------------------------Writing the EGF Data-------------------------------------------
    ##################################################################################################
    
    write_eeg(filename, data, Fs_EGF)
    

In [4]:
def get_remap_chan(chan_num):
    """There is re-mapping, thus to get the correct channel data, you need to incorporate re-mapping
    input will be a channel from 1 to 64, and will return the remapped channel"""
    
    remap_channels = np.array([32, 33, 34, 35, 36, 37, 38, 39, 0, 1, 2, 3, 4, 5, 
                      6, 7, 40, 41, 42, 43, 44, 45, 46, 47, 8, 9, 10, 11, 
                      12, 13, 14, 15, 48, 49, 50, 51, 52, 53, 54, 55, 16, 17, 
                      18, 19, 20, 21, 22, 23, 56, 57, 58, 59, 60, 61, 62, 63, 
                      24, 25, 26, 27, 28, 29, 30, 31])
    
    return remap_channels[chan_num-1]

def get_lfp_bytes(iterations):
    """This function works, but the strategy I use didn't. I was going to slice the bytearray like you would
    a numpy array but that didn't work"""
    data_byte_len = 384
    indices = np.arange(data_byte_len)
    indices = np.tile(indices, (1, iterations))
    indices = indices.flatten()

    offset_indices = np.arange(iterations)
    offset_indices = offset_indices*432 + 32
    offset_indices = np.tile(offset_indices,(384,1))
    offset_indices = offset_indices.flatten(order='F')
    
    return indices + offset_indices
    
def get_lfp_indices(iterations):
    data_byte_len = 192
    indices = np.arange(data_byte_len)
    indices = np.tile(indices, (1, iterations))
    indices = indices.flatten()

    offset_indices = np.arange(iterations)
    offset_indices = offset_indices*213 + 13
    offset_indices = np.tile(offset_indices,(192,1))
    offset_indices = offset_indices.flatten(order='F')
    
    return indices + offset_indices

    
def find_n(iterations, optimal=1000):
    n = optimal
    
    while True:
        if iterations % n == 0:

            return n
        else:
            n -= 1
    return 'abort'

    
def get_channel_bytes(channel_number, samples):
    """This will get the indices of the data if it is just the lfp data (not the bytes header bytes or trailing bytes)"""
    remap_channel = get_remap_chan(channel_number)
    
    indices_scalar = np.multiply(np.arange(samples), 64)
    sample_indices = indices_scalar + np.multiply(np.ones(samples), remap_channel)
    
    #return np.array([remap_channel, 64 + remap_channel, 64*2 + remap_channel])
    return (indices_scalar + np.multiply(np.ones(samples), remap_channel)).astype(int)


def get_channel_from_tetrode(tetrode):
    """This function will take the tetrode number and return the Axona channel numbers
    i.e. Tetrode 1 = Ch1 -Ch4, Tetrode 2 = Ch5-Ch8, etc"""
    tetrode = int(tetrode) # just in case the user gave a string as the tetrode
    
    return np.arange(1,5) + 4*(tetrode-1)

In [5]:
def CWTGaborWavelet(data, Fs, f_min=0.1, f_max=500, FreqSeg=None, StDevCycles=3, TimeStep=None,
                magnitudes=True, SquaredMag=False):
    """
    Will perform the continuous wavelet transform using the Gabor Wavelet
    
    Fourier transform is the most common tool for analyzing frequency properties, however the 
    time-evolution of the resulting frequencies are not reflected in the Fourier transform 
    (not directly). Essentially, it also does represent abrupt changes in the waveform efficiently. 
    The fourier transform represents data as a sum of sine waves which are not localized in
    time or space, and oscillate forever. Thus we will be using wavelets in order to keep
    both the time and frequency aspects of the signal (and better handle the transient signals).
    
    Wavelet is a rapidly decaying wave-like oscillation with a mean of zero. There are many
    types of wavelets: Morlet, Daubechies, Mexican Hat, etc. 
    
    The function that is going to be transformed is first multiplied by the Gaussian function,
    g(x), then is transformed using a Fourier Transform.
    
    Gaussian function, g(x) = (1/((σ^2 * π) ^(1/4))) * e^(-t^2 / (2*σ^2))
    Gabor Wavelet, ψ(t) = g(t)*e^(iηt)
    
    Guassian envelope windows are preferred as the same type of function would exist in either
    domain (if we did rectangular envelopes, the conjugate domain would have a sinx/x function).
    
    
    Continuous Wavelet Transform, C(s, t) = 1/sqrt(s)∫ f(t)*ψ((t-τ)/s), where s is scale,
    τ is translation, and η is angular frequency at scale s,
    
    σ = 6/η in order to satisfy the admissibility condition
    
    
    inputs:
    data: data to perform the trasnform on
    Fs: sampling frequency (Hz) of the collected data
    f_min: minimum frequency (Hz) to include in the resulting transform
    f_max: maximum frequency (Hz) to include in the resulting trasnform
    FreqSeg: the number of frequency segments to calculate
    StDevCycles: in the wavelet transform, the scale corresponding to each frequency defines 
    the gaussian's standard dev used for calculating the transform. This parameter defines
    the number of cycles you want to include in the transform at every frequency
    TimeStep: This step value will determine the downsampling of the time value, 
    default is None, in which case all time values are preserved
    Magnitudes: if True (default), it will return the magnitudes of the coefficients, False will 
    return analytic values (complex values)
    SquaredMag: 
    
    """
    
    if FreqSeg is None:
        FreqSeg = Fs
    
    if f_min >= f_max:
        print("The minimum frequency is larger than the maximum frequency, switching values")
        temp_values = (f_min, f_max)
        f_max, f_min = temp_values

    if f_max < 0 or f_min < 0:
        print("Inappropriate frequency value provided (does not accept negative values)")

    # checking that the max frequency is appropriate
    if f_max > Fs/2:
        f_max = Fs/2

    # checking that the minimum frequency is appropriate
    if f_min == 0:
        f_min = 0.1

    # if an improper segment value was provided, fix it
    if FreqSeg <= 0:
        FreqSeg = np.around(f_max - f_min)

    # calculate the frequency values

    step = (f_max - f_min) / (FreqSeg - 1)

    freqs = MatlabNumSeq(f_min, f_max, step).reshape((1, -1))
    # freqs = np.arange(f_min, f_max+step-f_min, step)

    # flip the matrix
    freqs = np.fliplr(freqs)

    if len(data) % 2 == 0:
        # need an odd number of data points
        data = data[:-1]

    t = np.arange(len(data))/Fs  # creating the time array

    n = len(t)  # number of data points
    n_half = int(np.floor(n/2) + 1)

    # creating an array from 0 -> 2 pi
    WAxis = np.multiply(np.arange(n), (2*np.pi / n)) 

    WAxis *= Fs
    WAxis_half = WAxis[:n_half]

    if TimeStep is None:
        SampAve = 1  # no timestep was given, keep all the time values
    else:
        SampAve = np.around(TimeStep*Fs)  #

        if SampAve < 1:
            SampAve = 1

    SampAveFilt = np.array([])
    if SampAve > 1:
        # down sample the time 
        #IndSamp = np.arange(0, len(t), SampAve)
        IndSamp = MatlabNumSeq(0, len(t)-1, SampAve)
        t = t[IndSamp]
        SampAveFilt = np.ones(SampAve, 1)

    data_fft = fftw.fft(data)  # transforming the input signal to the S domain
    #data_fft = conj_nonzeros(data_fft)

    GaborWT = np.zeros((freqs.shape[1], len(t)), dtype=np.complex)

    FreqInd = -1

    # reshaped freqs so that the freq value would take on a single value, not the entire array
    for freq in freqs.reshape((-1, 1)):
        # Calculating each frequency
        StDevSec = (1/freq) * StDevCycles

        WinFFT = np.zeros((n, 1))
        WinFFT[:n_half] = np.exp(-0.5 * np.power(
                WAxis_half - (2 * np.pi * freq), 2) * (StDevSec **2)
                                ).reshape((-1,1))
        WinFFT = np.multiply(WinFFT, (np.sqrt(n) / np.linalg.norm(WinFFT,2)))
        # need to convert to complex otherwise imaginary numbers will be discarded
        WinFFT = np.asarray(WinFFT, dtype=np.complex).flatten()  

        FreqInd += 1

        if SampAve > 1:

            GaborTemp = np.zeros(len(data_fft) + SampAve - 1, 1)
            GaborTemp[SampAve-1:] = fftw.ifft(np.multiply(data_fft, WinFFT)) / sqrt(StDevSec)

            if magnitudes:
                # return only the magnitudes
                GaborTemp = np.absolute(GamborTemp)

            if SquaredMag:
                # return the squared magnitude
                if not magnitudes:
                    GaborTemp = np.absolute(GamborTemp)
                GaborTemp = GaborTemp**2

            GaborTemp[:SampAve-1] = np.flipud(GaborTemp[SampAve, 2 * SampAve])
            GaborTemp = np.divide(signal.lfilter(SampAveFilt, 1, GaborTemp), SampAve)
            GaborTemp = GaborTemp[SampAve-1:]

            GaborWt[FreqInd, :] = GaborTemp[IndSamp]
        else:
            GaborWT[FreqInd, :] = fftw.ifft(np.multiply(data_fft, WinFFT)) / np.sqrt(StDevSec)

    if SampAve > 1:
     
        return GaborWT, t, freqs

    if magnitudes:
  
        GaborWT = np.absolute(GaborWT)

    if SquaredMag:
        if not magnitudes:
            GaborWT = np.absolute(GaborWT)
        GaborWT = GaborWT**2
    
    # we flipped the GaborWT to have it in the same format as our
    # Stockwell Transform method
    return np.flipud(GaborWT), t, np.fliplr(freqs)
    
def MatlabNumSeq(start, stop, step):
    """In Matlab you can type: 
    
    start:step:stop and easily create a numerical sequence
    """
    
    '''np.arange(start, stop, step) works good most of the time
    
    However, if the step (stop-start)/step is an integer, then the sequence
    will stop early'''
    
    seq = np.arange(start,stop+step,step)
    
    if seq[-1] > stop:
        seq = seq[:-1]
        
    return seq
    

def data2wavelet(data):
    """
    
    """
    
    first_half_ind = int(np.around(len(data)*0.5) + 1)  # indices of the first half of the data

    initial_window = data[:first_half_ind+1]  # uses the first half of the data as the initial window
    end_window = data[-2-first_half_ind:]  # defines the end window

    first = data[0]  # the first value
    last = data[-1]  # defines the last value

    initial_window = np.subtract(initial_window, first)  # wavelets start at 0
    end_window = np.subtract(end_window, last)  # wavelets end with a 0

    initial_window = np.flipud(-initial_window) + first
    end_window = np.flipud(-end_window) + last

    initial_window = initial_window[:-1]
    end_window = end_window[1:]
    
    return np.concatenate((initial_window, data, end_window)), initial_window, end_window

In [6]:
def get_bin_data(bin_filename, channels=None, tetrode=None):
    """This function will be used to acquire the actual lfp data given the .bin filename,
    and the tetrode or channels (from 1-64) that you want to get"""

    if tetrode is not None:
        channels = get_channel_from_tetrode(tetrode)
    else:
        channels = np.array(channels)  # just in case it isn't an np.array

    bytes_per_iteration = 432

    with open(bin_filename, 'rb') as f:
        # pass
        with contextlib.closing(mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)) as m:
            num_iterations = int(len(m)/bytes_per_iteration)

            data = np.ndarray((num_iterations,), (np.int16, (1,192)), m, 32, (bytes_per_iteration,)).reshape((-1, 1)).flatten()
            data = samples_to_array(data, channels=channels.tolist())

    return data

In [7]:
active_eeg_channels = get_active_eeg(set_filename)
print(active_eeg_channels)
active_eeg_channel_numbers = np.asarray(
    list(active_eeg_channels.values())) + 1  # makes the eeg channels from 1-> 64

print(active_eeg_channel_numbers)

{1: 16, 2: 17, 3: 18, 4: 19}
[17 18 19 20]


In [8]:
channel = active_eeg_channel_numbers[0]
Fs = 48e3
Fs_EEG = 250

data = get_bin_data(bin_filename, channels=[channel])

print('done')

done


In [9]:
# notch filter the data
data = sp.Filtering().notch_filt(data, Fs, freq=60, band=10,
                                 order=3)

In [10]:
data = sp.Filtering().iirfilt(bandtype='low', data=data, Fs=Fs, Wp=Fs_EEG / 2, order=6,
                                  automatic=0, Rp=3, As=60, filttype='butter', showresponse=0)

In [ ]:
def int16toint8(value):
    """Converts int16 data to int8"""
    value = np.rint(np.divide(value, 256))

    value[np.where(value > 127)] = 127
    value[np.where(value < -128)] = -128
    return value

data = data[:, 0::int(Fs / Fs_EEG)]

# ensuring the appropriate range of the values
data[np.where(data > 32767)] = 32767
data[np.where(data < -32768)] = -32768

data = int16toint8(data)  # converting from 16 bits to 8 bits,

In [ ]:
plt.plot(data[:2000])